In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
!git clone https://github.com/academy-dt/nlp-text-summarisation

In [ ]:
import os
os.chdir('./nlp-text-summarisation')

In [ ]:
!git submodule init
!git submodule update

In [ ]:
!pip install sklearn transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
test_dataset = '/content/drive/MyDrive/test_dataset'
test_data_chunk = '000'
test_data_path = f'{test_dataset}/test_{test_data_chunk}.bin'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

device = 'cuda:0'
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn").to(device)
pipe = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

In [ ]:
from generators import get_cnn_dm_both_generator

test_data_gen = get_cnn_dm_both_generator(test_data_path)
test_data = list(test_data_gen)

In [ ]:
import json

output = []
for article, abstract in test_data[:1]:
    try:
        tokens = tokenizer(article, return_tensors="pt", padding=True, truncation=True).to(device)
        outputs = model(**tokens)
        summary = tokenizer.decode(outputs, skip_special_tokens=True)
        output.append({
            'article': article,
            'abstract': abstract,
            'summary': summary
        })
    except Exception as ex:
        print(f'\t ERROR: {str(ex)}')

# with open(f'bart_output_{test_data_chunk}.json', 'w') as fout:
#     json.dump(output, fout, indent=2)